# Topic modeling (LDA)
- It implements latent dirichlet allocation (a popular topic modeling approach)
- The model uses collapsed gibbs sampling (a faster inference model for topic modeling)

It operates in two steps.

*A) Preparing data (integer encoding documents)*  

*B) Performing topic modeling on integer encoded documents*

In [124]:
# Its vanilla implementation of Topic modeling that only uses basic tools:
# json - to read from and write to files in json format 
# numpy - for faster matrix operations 
# pandas - to read csv data
# string - to only keep English letters, removing puntuations and other characters
# random - to generate random numbers for initializing Markov-chain monte carlo, and 
#           and during algorithm working to avoid local optima


import json
import numpy as np
import random
import pandas as pd
import string

# A) Preparing data (integer encoding documents)

1. Read textual data
2. Generate integer encoding
3. Storing intemediate data

**Working with integers (representing words or unique tokens is much faster than the word strings itself)**

*At the end, the integers would be reversed back to their respective words*

## 1. Reading textual data
- Read raw text from .txt file having document per line
- Separate into list of documents
- Tokenize

1.1 Clean text by removing punctuations and characters othen than English letters \
1.2 Convert to lower case \
1.3 Tokenize 

In [125]:
en_stopwords = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

def clean_text(text):
    clean_text = text.lower()
    # cleaning documents by removing unwanted characters
    clean_text = "".join([char for char in text if char in string.ascii_lowercase])
    # cleaning documents by stopwords
    clean_text = [word for word in text.split(" ") if word not in en_stopwords and len(word) > 2]
    return clean_text

1.4 Read data from the file 

In [126]:
# Read input data: titles of BBC articles available on the following link
# https://github.com/vahadruya/Capstone-Project-Unsupervised-ML-Topic-Modelling/blob/main/Input_Data/input.csv

with open('config.json', 'r') as file:
    configurations = json.load(file)
df = pd.read_csv(configurations["text-doc-path"])
rawdata = df["Title"].tolist()


# Tokenize sentences into words
tokenized_documents = []
for document in rawdata[:100]: # Considering only first 100 titles for the sake of demonstration
    document = clean_text(document)
    if len(document) > 2:
        tokenized_documents.append(document)
len(tokenized_documents)

100

## Configuration

`config.json`

The method provides the following configuration options to alter the behavior of the method.

**numTopics:** Number of topics to extract from the dataset. The default value is 3. However, it generally depends on the nature of the data. Keeping the number of topics too few can only have the topics focused on broader concepts and cannot identify the specific topics. While keeping the number of topics too high results in noisy (incoherent) topics. Therefore, a suitable number of topics depends on the data and the type of analysis required. The default value here is 3.

**numAlpha**: This hyperparameter helps in deciding the probabilities of topics in a document. A higher value (above 1) will have too many topics with similar probabilities i.e., when the intended purpose is to get more topics per document. However, keeping the value too low will have only few promiment topics with very high probabilities as compared to others. A lower value (below 1) is used when only fewer topics are needed per document. A lower numAlpha value pushes higher probabilities higher and lower probabilities further lower. While a higher numAlpha value introduces a high bias due to which all probabilities converges to similar values. In this method we are using numAlpha value 1.

**numBeta**: This hyperparameter helps in deciding the probabilities of words in a topic. a higher value (above 1) will have too many words with similar probabilities i.e., when the intended purpose is to have more words representing a topic. However, keeping the value too low will have only few most prominent words with very high probabilities as compared to others. Generally, considering the size of vocabulary in a dataset, this value is kept smaller to determine only the most relevant words. In this method we are using numBeta value 0.01.

**numGSIterations**: Its the number of iterations of the inference technique (collapsed Gibbs sampling). Due to random initialization, there are more words switching their topics in the earlier iterations that keeps dropping in the coming iterations i.e., approaching the equilibrium state. Keeping the number of numGSIterations higher ensures that the words have settled down in their respective topics. Alternately, the difference between two consecutive iterations is also used to avoid unnecessary iterations when the words have already settled. In this method we only use the numGSIterations with value 1000.

**wordsPerTopic**: The number of top words to represent a topic. In this method we are using 10 most prominent words for each topic. Due to polysemy, it is possible that a word exist in different topics with different neighboring words highlighting its context. 

**text-doc-path**: path of input (raw text) file
**integer-encoded-doc-path**: path of integer encoded file. It is the intermediate file that topic modeling use. 
**integer-word-dict**

## 2. Generate Integer encoding
It preserves both frequency and position related information. The process involves assigning each unique token a dedicated integer id, preserving it in a dictionary for later retrieval, while rewriting documents by replacing with with their integer ids.

It makes the operations a lot faster as numbers are much faster to read/store and compare as compared to strings. 

The integer ids will be replaced with their original words at the end using stored dictionary files

2.1 Generate integer encoded documents \
2.2 Generate word-integer index and integer index-word dictionaries

In [127]:
# Create a dictionary of unique tokens and assign integers
dictionary = {}
revdictionary = {}
index = 0

#tokenized_documents = [[word for word in doc if word not in esw] for doc in tokenized_documents]

for doc in tokenized_documents:
    for word in doc:
        if word not in dictionary.keys():
            dictionary[word] = index
            revdictionary[index] = word
            index += 1

# Replace words in sentences with their corresponding integers
encoded_documents = [[dictionary[word] for word in doc] for doc in tokenized_documents]

### 3. Storing intermediate data
The integer encoded documents are stored in files
the word-to-id and id-to-word dictionaries are also stored

*It will help to avoid these steps, each time topic modeling is performed under different settings*

In [128]:
toStr = ''
for endoc in encoded_documents:
    toStr = toStr + '\t'.join(str(item) for item in endoc)
    toStr = toStr + '\n'
toStr = toStr[:-2]
file = open('data/integer-encoded-data.txt', 'w')
file.write(toStr)
file.close()

#write dictionary to file
file = open('data/dictionary.json', 'w')
file.write(json.dumps(dictionary))
file.close()
file = open('data/revdictionary.json', 'w')
file.write(json.dumps(revdictionary))
file.close()

# B) Topic Modeling (LDA)
- It identifies the hidden thematic structures within the documents and represent them as latent topics.
- Each document is a mixture of all possible topics with varying probabilities
- Each topic is a mixture of all vocabulary of the dataset with varying probabilities

*Setting random seeds*

In [129]:
# For reproducible results
random.seed(41)  # For Python random
np.random.seed(41)  # For NumPy random

**LDA class**
main functions are:
1. Markov chain monte carlo initialization (giving the model a random inital state, expecting the model
    to converge for higher number of iterations.
2. Collapsed gibbs sampling inference: in each iteration \
   2.1 Iterates through all documents, all tokens/words in each document \
   2.2 For for each token computes its most suitable topic, given the current status of the model \
   2.3 Updates new topic if different from current topic, associated estimates update, so does the model state \
3. Estimate document-topic distribution from the final state of the model 
4. Estimate topic-word distribution (organized in decreasing order of probabilities) from the final state of the model
5. Other utility functions

In [140]:
# The class implements topic modeling (Latent dirichlet allocation) algorithm using collapsed gibbs sampling as in inference. 
class LDA:
    # topics to extract from the data (Components)
    _numTopics = None
    # vocabulary (unique words) in the dataset
    _arrVocab = None
    #size of vocabulary (count of unique words)
    _numVocabSize = None
    # dataset
    _arrDocs = []
    # dataset size (number of documents)
    _numDocSize = None
    # dirichlet prior (document to topic prior)
    _numAlpha = None
    # dirichlet prior (topic to word prior)
    _numBeta = None
    _ifScalarHyperParameters = True
    # Gibb sampler iterations
    _numGSIterations = None
    # The iterations for initial burnin (update of parameters)
    _numNBurnin = None
    # The iterations for continuous burnin (update of parameters)
    _numSampleLag = None
    
    
    
    # The following attributes are for internal working
    __numTAlpha = None  
    __numVBeta = None   
    __arrTheta = None
    __arrThetaSum = None
    __arrPhi = None
    __arrPhiSum = None
    __arrNDT = None
    __arrNDSum = []
    __arrNTW = None
    __arrNTSum = []
    __arrZ = []
    
    # for alpha to be a list, its size must be equal to the size of the dataset, has value for each doc
    # for beta to be a list, its size must be equal to the number of topics, has value for each topic  
    def __init__(self, numTopics = 2, numAlpha = 1.0, numBeta = 0.01, 
                 numGSIterations = 1000, numNBurnin = 50, numSampleLag = 20, 
                 wordsPerTopic = 10):
        self._numTopics = configurations["numTopics"]
        self._numAlpha = configurations["numAlpha"]
        self._numBeta = configurations["numBeta"]
        self._numGSIterations = configurations["numGSIterations"]
        self._numNBurni = configurations["numNBurnin"]
        self._numSampleLag = configurations["numSampleLag"]
        self.__wordsPerTopic = configurations["wordsPerTopic"]
            
    #load data as integer encoding of words in a sequence (no padding or truncation)
    def getData(self, path):
        file = open(path, 'r')
        rawData = file.read()
        file.close()
        self.__loadData(rawData)
        self.__loadVocab()
        self.__prepareCollections()

    #load docs and docSize from the dataset
    def __loadData(self, rawData):
        rows = rawData.split('\n')
         
        #read dataset as documents of words IDs
        for row in rows:
            swordlist = row.split('\t')
            swordlist = list(filter(None, swordlist))   #remove empty items from list
            if len(swordlist) > 0:
                iwordlist = [eval(w) for w in swordlist]    
                self._arrDocs.append(iwordlist)

        # determine dataset size
        self._numDocSize = len(self._arrDocs)
        
        
    #Determine unique words (vocabulary) and count of unique words (vocabSize)    
    def __loadVocab(self):
        #determine unique vocabulary
        uniqueWords = []
        for doc in self._arrDocs:
            for word in doc:
                if word not in uniqueWords:
                    uniqueWords.append(word)
        self._arrVocab = uniqueWords
        self._numVocabSize = len(self._arrVocab)    

    def __prepareCollections(self):
        self.__arrNDSum = np.array([0] * self._numDocSize)
        self.__arrTheta = np.array([[0] * self._numTopics] * self._numDocSize)
        self.__arrThetasum = np.array([[0] * self._numTopics] * self._numDocSize)
        self.__arrNDT = np.array([[0] * self._numTopics] * self._numDocSize)
        
        self.__arrNTSum = np.array([0] * self._numTopics)
        self.__arrPhi = np.array([[0] * self._numVocabSize] * self._numTopics)
        self.__arrPhisum = np.array([[0] * self._numVocabSize] * self._numTopics)
        self.__arrNTW = np.array([[0] * self._numVocabSize] * self._numTopics)

        #Assign values to parameters based on hyper-parameters
        self.__numTAlpha = self._numTopics*self._numAlpha  
        self.__numVBeta = self._numVocabSize*self._numBeta   

        
        for d in range(0, self._numDocSize):
            rowOfZeros = [0] * len(self._arrDocs[d])
            self.__arrZ.append(rowOfZeros)
                
    # Initialize first markov chain randomly
    def randomMarkovChainInitialization(self):
        
        for d in range(self._numDocSize):
            wta = []                        #wta - word topic assignment
            doc = self._arrDocs[d]
            for ind in range(len(doc)): 
                randtopic = random.randint(0, self._numTopics - 1)      # generate a topic number at random
                self.__arrZ[d][ind] = randtopic
                self.__arrNDT[d][randtopic] += 1
                self.__arrNDSum[d] += 1
                wordid = self._arrDocs[d][ind]
                self.__arrNTW[randtopic][wordid] += 1
                self.__arrNTSum[randtopic] += 1
            
    
    #Inference (Collapsed Gibbs Sampling)
    def gibbsSampling(self):
        tAlpha = self._numAlpha * self._numTopics
        vBeta = self._numBeta * self._numVocabSize            
                    
        for it in range(self._numGSIterations):
            for d in range(self._numDocSize):
                dsize = len(self._arrDocs[d])
                for ind in range(dsize):
                    # remove old topic from a word instance
                    oldTopic = self.__arrZ[d][ind]
                    wordid = self._arrDocs[d][ind]
                    self.__arrNDT[d][oldTopic] -= 1
                    self.__arrNDSum[d] -= 1
                    self.__arrNTW[oldTopic][wordid] -= 1
                    self.__arrNTSum[oldTopic] -= 1   

                    # find a new more appropriate tpoic for the word instanc as per current state of the model
                    prob = [0] * self._numTopics
                    
                    for t in range(self._numTopics):
                        prob[t] = ((self.__arrNDT[d][t] + self._numAlpha) / (self.__arrNDSum[d] + tAlpha)) * \
                            (self.__arrNTW[t][wordid] + self._numBeta) / (self.__arrNTSum[t] + vBeta)
                    
                    #cumulate multinomial
                    cdf = prob
                    for x in range(1, len(cdf)):
                        cdf[x] += cdf[x-1]
                    
                    cutoff = random.random() * cdf[-1]
                    newTopic = 0
                    for i in range(len(cdf)):
                        if cdf[i] > cutoff:
                            newTopic = i
                            break
                    #update as per new topic
                    self.__arrZ[d][ind] = newTopic
                    self.__arrNDT[d][newTopic] += 1
                    self.__arrNDSum[d] += 1
                    self.__arrNTW[newTopic][wordid] += 1
                    self.__arrNTSum[newTopic] += 1
                
    def getTopicsPerDocument(self):
        results = ''
        results += "***Topics per Document***\n"
        for d in range(self._numDocSize):
            results += "Document " + str(d) + ":\n"
            for t in range(self._numTopics):
                val = (self.__arrNDT[d][t]+self._numAlpha)/(self.__arrNDSum[d]+self.__numTAlpha)
                results += "Topic " + str(t) + ":" + str(val) + '\t'
            results += '\n'
        print(results)
        file = open('data/output-data/document-topic-distribution.txt', 'w')
        file.write(results)
                    
   
    def getWordsPerTopic(self, revdictionary):
        results = "***Words per Topic***\n"
        
        for t in range(self._numTopics):
            results += "\nTopic " + str(t) + ":"
            #flag = 0
            wpt = {}
            for v in range(self._numVocabSize):
                val = (self.__arrNTW[t][v]+self._numBeta)/(self.__arrNTSum[t]+self.__numVBeta)
                wpt[revdictionary[str(v)]] = float(val)
             #   flag += 1
             #   if flag == self.__wordsPerTopic:
             #       break
            results += '\n'
            wpt = sorted(wpt.items(), key=lambda x: x[1], reverse=True)[:self.__wordsPerTopic]
            for item in wpt:
                results += str(item)
        file = open('data/output-data/document-topic-distribution.txt', 'w')
        file.write(results)
        print(results)
    
    def printall(self):
        print("topics: ", self._numTopics)
        print("dataset: ", self._arrDocs)
        print("dataset size: ", self._numDocSize)
        print("vocab: ", self._arrVocab)
        print("vocab size: ", self._numVocabSize)
        print("ndt: ", self.__arrNDT)
        print("ndsum: ", self.__arrNDSum)
        print("ntw: ", self.__arrNTW)
        print("ntsum: ", self.__arrNTSum)
        print("z: ", self.__arrZ)

## Run the model

In [141]:
if __name__ == "__main__":
    lda = LDA()
    lda.getData(configurations["integer-encoded-doc-path"])
    lda.randomMarkovChainInitialization()
    lda.gibbsSampling()

## Results

Topic modeling has two important results
1. **Latent topics** identified in the corpus. Each topic is represented by top most presentable words for that topic. Its similar to clustering in the sense that the words are grouped as topics and labeled unintuitively as topic 0, topic 1 etc. However, unlike clustering, the words have probabilities of relevance to the other words of the topic. Using these probabilities, only the top few words (10 or 20) are used to represent a topic. Therefore, it is also called *word topic distribution*. 

2. **Topics in documents** are the probabilities of topics within each document. A general conception is that a document is not about entirely about a single topic and instead has different percentages of multiple topics. The topics in documents provide the probabilities of each topic in each document.

To observe the output generated by this method closely, please goto `data/output-data/` folder having `word-topic-distribution.txt` and `document-topic-distribution.txt` 

**words distribution per topic** \
The three latent topics determind from this dataset are labeled as Topic 0, Topic 1, and Topic 2. \
*Topic 0:* Represents profit, production and growth in economy \
*Topic 1:* Represents trade and and deals of fuel and BMW mentioning Germany and France \
*Topic 2:* Represents jobs, Yukos firm, India and Japan

These three topics gives a general idea of the topics covered in the data.

*Topic word distribution*

In [142]:
with open(configurations["integer-word-dict"], 'r') as file:
    revdictionary = json.load(file)
lda.getWordsPerTopic(revdictionary)

***Words per Topic***

Topic 0:
('profits', 0.028118645256293383)('profit', 0.028118645256293383)('firm', 0.028118645256293383)('Japan', 0.028118645256293383)('Japanese', 0.021106514269686554)('2004', 0.014094383283079725)('prices', 0.014094383283079725)('Parmalat', 0.014094383283079725)('recession', 0.014094383283079725)('wine', 0.014094383283079725)
Topic 1:
('hits', 0.03138901071361443)('jobs', 0.025123739114090594)('growth', 0.025123739114090594)('India', 0.018858467514566754)('new', 0.018858467514566754)('oil', 0.018858467514566754)('trade', 0.012593195915042914)('takeover', 0.012593195915042914)('lifts', 0.012593195915042914)('production', 0.012593195915042914)
Topic 2:
('economy', 0.0381320982171182)('deal', 0.0381320982171182)('fuel', 0.0317873231393947)('Yukos', 0.02544254806167121)('gets', 0.02544254806167121)('German', 0.019097772983947717)('sales', 0.019097772983947717)('BMW', 0.019097772983947717)('rise', 0.012752997906224223)('$280bn', 0.012752997906224223)


**Topic distribution per document** \
Each document talks about the topics identified to different extent. For example, document 0 can be 45% topic 0, 45% topic 1 and 10% topic 2. 

Therefore, it is important to know which documents are dominated by which topics, so that if a reader is interested in knowing particularly about topic 1 they can only read the documents where topic 1 is the major topic.

In [143]:


lda.getTopicsPerDocument()



***Topics per Document***
Document 0:
Topic 0:0.125	Topic 1:0.75	Topic 2:0.125	
Document 1:
Topic 0:0.125	Topic 1:0.375	Topic 2:0.5	
Document 2:
Topic 0:0.375	Topic 1:0.125	Topic 2:0.5	
Document 3:
Topic 0:0.5	Topic 1:0.125	Topic 2:0.375	
Document 4:
Topic 0:0.2857142857142857	Topic 1:0.42857142857142855	Topic 2:0.2857142857142857	
Document 5:
Topic 0:0.14285714285714285	Topic 1:0.14285714285714285	Topic 2:0.7142857142857143	
Document 6:
Topic 0:0.125	Topic 1:0.5	Topic 2:0.375	
Document 7:
Topic 0:0.125	Topic 1:0.5	Topic 2:0.375	
Document 8:
Topic 0:0.25	Topic 1:0.375	Topic 2:0.375	
Document 9:
Topic 0:0.42857142857142855	Topic 1:0.42857142857142855	Topic 2:0.14285714285714285	
Document 10:
Topic 0:0.4444444444444444	Topic 1:0.2222222222222222	Topic 2:0.3333333333333333	
Document 11:
Topic 0:0.14285714285714285	Topic 1:0.2857142857142857	Topic 2:0.5714285714285714	
Document 12:
Topic 0:0.5	Topic 1:0.125	Topic 2:0.375	
Document 13:
Topic 0:0.2222222222222222	Topic 1:0.1111111111111111	T

*Document topic distribution*


*print all details:*
- Integer encoded dataset
- Final state of the model

In [144]:
# prints everything - for debugging
#lda.printall()